In [2]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image

from torchvision import transforms

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
IMG_H = 28
IMG_W = 28
IMG_C = 3

In [5]:
# Transform to resize images
motion_transform = transforms.Compose([
    transforms.Resize((IMG_H, IMG_W)),
    transforms.ToTensor(),
])

In [6]:
# Custom Dataset class to read the data from the excel file and image directory
class MotionDataset(Dataset):
    def __init__(self, xlsx_file, image_dir, transform=None):
        """
        Args:
            xlsx_file (string): Path to the excel file with annotations.
            image_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data_info = pd.read_excel(xlsx_file)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_info)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.image_dir,
                                str(self.data_info.iloc[idx]['PrimaryImageFilename']))
        image = Image.open(img_name).convert('RGB')

        # 'PGC1' is the target label
        label = self.data_info.iloc[idx]['PGC1']-1

        if self.transform:
            image = self.transform(image)

        return image, label

In [7]:
dataset = MotionDataset(xlsx_file='cleaned_product_list.xlsx', image_dir='/home/hice1/rlopez76/scratch/motion_dataset', transform=motion_transform)
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32) # need to look into other parameters
test_loader = DataLoader(test_dataset, batch_size=32)

train_N = len(train_loader.dataset)
test_N = len(test_loader.dataset)

In [8]:
N_CLASSES = 8
KERNEL_SIZE = 3
FLATTENED_IMG_SIZE = IMG_H * IMG_W * IMG_C

model = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=25, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(25),
        nn.ReLU(),        
        nn.MaxPool2d(2, stride = 2),
        nn.Conv2d(in_channels=25, out_channels=50, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(50),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.MaxPool2d(2, stride = 2),
        nn.Conv2d(in_channels=50, out_channels=75, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(75),
        nn.ReLU(),
        nn.Dropout(0.2), 
        nn.MaxPool2d(2, stride = 2),
        nn.Flatten(), 
        nn.Linear(675, 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.ReLU(),
        nn.Linear(512, N_CLASSES)
)

In [9]:
model = model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

def save_checkpoint(model, optimizer, epoch, loss, save_path):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }, save_path)
    
def load_checkpoint(model, optimizer, load_path):
    checkpoint = torch.load(load_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    
    return model, optimizer, epoch, loss

def get_batch_accuracy(output, y, N):
    pred = output.argmax(dim = 1, keepdim = True)
    correct = pred.eq(y.view_as(pred)).sum().item()
    acc = correct / N
    return acc

def train():
    loss = 0
    accuracy = 0

    model.train()
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)
#         print("x shape", x.shape)
        
#         firstx = x
        
#         layer = nn.Conv2d(in_channels=3, out_channels=25, kernel_size=3, stride=1, padding=1)
#         out_tensor = layer(x)
        
#         layer = nn.BatchNorm2d(25)
#         out_tensor = layer(out_tensor)
        
#         layer = nn.ReLU()        
#         out_tensor = layer(out_tensor)
                        
#         layer = nn.MaxPool2d(2, stride = 2)
#         out_tensor = layer(out_tensor)
            
#         layer = nn.Conv2d(in_channels=25, out_channels=50, kernel_size=3, stride=1, padding=1)
#         out_tensor = layer(out_tensor)
        
#         layer = nn.BatchNorm2d(50)
#         out_tensor = layer(out_tensor)
            
#         layer = nn.ReLU()    
#         out_tensor = layer(out_tensor)
        
#         layer = nn.Dropout(0.2)
#         out_tensor = layer(out_tensor)
        
#         layer = nn.MaxPool2d(2, stride = 2)
#         out_tensor = layer(out_tensor)        
        
#         layer = nn.Conv2d(in_channels=50, out_channels=75, kernel_size=3, stride=1, padding=1)
#         out_tensor = layer(out_tensor)
        
#         layer = nn.BatchNorm2d(75)
#         out_tensor = layer(out_tensor)
            
#         layer = nn.ReLU()
#         out_tensor = layer(out_tensor)
            
#         layer = nn.Dropout(0.2)    
#         out_tensor = layer(out_tensor)
                
#         layer = nn.MaxPool2d(2, stride = 2)
#         out_tensor = layer(out_tensor)
        
#         layer = nn.Flatten()        
#         out_tensor = layer(out_tensor)
                
#         layer = nn.Linear(675, 512)
#         out_tensor = layer(out_tensor)
                
#         layer = nn.ReLU()
#         out_tensor = layer(out_tensor)
                
#         layer = nn.Dropout(0.3)
#         out_tensor = layer(out_tensor)
        
#         layer = nn.ReLU()
#         out_tensor = layer(out_tensor)
        
#         layer = nn.Linear(512, N_CLASSES)
#         out_tensor = layer(out_tensor)
        
#         print("out_dimensions", out_tensor.shape)
        
        output = model(x)
        optimizer.zero_grad()
        batch_loss = loss_fn(output, y)
        batch_loss.backward()
        optimizer.step()
        
        loss += batch_loss.item()
        accuracy += get_batch_accuracy(output, y, train_N)

    print("Training Loss", loss)
    print("Training Accuracy", accuracy)

def validate():
    loss = 0
    accuracy = 0

    model.eval()
    with torch.no_grad():
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)
            output = model(x)
            loss += loss_fn(output, y).item()
            accuracy += get_batch_accuracy(output, y, test_N)
    
    print("Testing Loss", loss)
    print("Testing Accuracy", accuracy)

In [ ]:
%%time

if os.path.isfile("checkpoint.tar"):
    model, optimizer, epoch, loss = load_checkpoint(model, optimizer, "checkpoint.tar")

N_EPOCHS = 20
for N in range(N_EPOCHS):
    print(f"Epoch {epoch + 1}")
    train()
    validate()
    print("\n")

Epoch 20
Training Loss 35.183555433526635
Training Accuracy 0.940501043841336
Testing Loss 7.879574125632644
Testing Accuracy 0.9526791927627003


Epoch 20
Training Loss 24.957764111924917
Training Accuracy 0.9577244258872643
Testing Loss 9.632865861058235
Testing Accuracy 0.9318023660403618


Epoch 20
Training Loss 19.64739220123738
Training Accuracy 0.9672929714683361
Testing Loss 8.402048360556364
Testing Accuracy 0.9498956158663884


Epoch 20
Training Loss 17.11409496096894
Training Accuracy 0.9712943632567852
Testing Loss 9.624446269124746
Testing Accuracy 0.940153096729297


Epoch 20
Training Loss 15.575661132577807
Training Accuracy 0.9712943632567845
Testing Loss 18.21726930886507
Testing Accuracy 0.8803061934585943


Epoch 20
Training Loss 15.656229888903908
Training Accuracy 0.9719902574808628
Testing Loss 12.337572522461414
Testing Accuracy 0.9116214335421015


Epoch 20
Training Loss 14.250629106303677
Training Accuracy 0.9751217814892136
Testing Loss 10.203531759791076
Test

In [1]:
save_checkpoint(model, optimizer, 19, 20.49, "checkpoint.tar")

NameError: name 'save_checkpoint' is not defined